# Interactive annotation with Pixano [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixano/pixano/blob/main/notebooks/models/interactive_annotation.ipynb)

This notebook allows you to pre-compute embeddings for a whole Pixano dataset using a deep learning model, and convert the model to ONNX.

You can then interactively annotate your dataset using that model in the Pixano app.

## 1. Setting up

### Install dependencies

This notebook requires installing `pixano` and `pixano-inference`.

If you are running this notebook on your computer, we strongly recommend creating a virtual environment for using Pixano like so:

```shell
conda create -n pixano_env python=3.10
conda activate pixano_env
```

```shell
pip install pixano
pip install pixano-inference
```

If you are running this notebook in Google Colab, run the cell below to install `pixano` and `pixano-inference`.

In [ ]:
try:
    ENV = "colab"
    %pip install pixano
    %pip install pixano-inference
except:  # noqa: E722
    ENV = "jupyter"

You will also need to install SAM and MobileSAM separately:

In [ ]:
%pip install segment-anything@git+https://github.com/facebookresearch/segment-anything
%pip install mobile-sam@git+https://github.com/ChaoningZhang/MobileSAM

### Load dependencies

In [ ]:
from pathlib import Path

from pixano_inference import github

from pixano.app import App

### Download checkpoints
- SAM:
    - ViT-H checkpoint: [sam_vit_h_4b8939.pth](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth)
    - ViT-L checkpoint: [sam_vit_l_0b3195.pth](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth)
    - ViT-B checkpoint: [sam_vit_b_01ec64.pth](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth)
- MobileSAM:
    - ViT-T checkpoint: [mobile_sam.pt](https://github.com/ChaoningZhang/MobileSAM/raw/master/weights/mobile_sam.pt)
    
## 2. Computing embeddings

### Select a model

- SAM
```python
model = github.SAM(
    checkpoint_path=Path("my_datasets/models/sam_vit_b_01ec64.pth"),
    size="b"
)
```
- MobileSAM
```python
model = github.MobileSAM(
    checkpoint_path=Path("my_datasets/models/mobile_sam.pt"),
)
```

By default, `device` is equal to `"cuda"` for SAM and `"cpu"` for MobileSAM.

If you would like to run the model on a different device, please add `device="cuda"` or `device="cpu"` in the arguments in the cell below.

In [ ]:
model = github.SAM(
    checkpoint_path=Path("my_datasets/models/sam_vit_h_4b8939.pth"), size="h"
)

### Select a Pixano format dataset

If you haven't already, please refer to the [dataset notebooks](../datasets/) for information on how to import your dataset to Pixano format.

In [ ]:
library_dir = Path("my_datasets/")
dataset_dir = library_dir / "coco_instances"

views = ["image"]
splits = []

### Compute embeddings

In [ ]:
help(model.process_dataset)

In [ ]:
dataset = model.process_dataset(
    dataset_dir=dataset_dir,
    process_type="segment_emb",
    views=views,
    splits=splits,
    batch_size=2,
)

## 3. Exporting model to ONNX

To use your model inside Pixano app, you will need to convert it to ONNX.

In [ ]:
model.export_to_onnx(library_dir)

## 4. Annotating the dataset

With the generation complete, you can now browse your dataset with the Pixano app and start annotating.

You can stop the app by restarting the notebook.

In [ ]:
app = App(library_dir)

In [ ]:
app.display()